In [1]:
from Gapfilling import load_query_model, load_template_models
import re
import matplotlib as plt
import pandas as pd
import os
import cobra
from cobra.io import read_sbml_model
import gurobipy
from cobra.flux_analysis import gapfill

In [2]:
from cobra.flux_analysis.gapfilling import GapFiller

#### NEW FUNCTIONS

In [4]:
def gapfilling(model, template, integer_threshold, it = 1):
    """
    Calls GapFiller class from cobrapy.
    """
    # gurobi max integer threshold value -> 1e-9
    gapfiller = GapFiller(model, template, integer_threshold = integer_threshold,
                         demand_reactions = False, exchange_reactions=False)
    # not valid for gurobi:
    # gapfiller.model.solver.configuration.tolerances.feasability = 1e-10
    return gapfiller.fill(iterations = it)

def add_exchange_reactions(model, template):
    """
    Adds all exchange reactions from a template matching model metabolites. 
    """
    EX = [template.reactions.get_by_id(i.id) for i in template.reactions if i.id.startswith("EX_")]
    for reaction in EX:
        if reaction not in model.reactions and str(list(reaction.metabolites.keys())[0]) in [i.id for i in model.metabolites]:
            model.add_reaction(reaction.copy())
    return model

In [6]:
# add_exchange_reactions test
x = read_sbml_model("BiGG_files/e_coli_core.xml")
x

Name,e_coli_core
Memory address,0x07f9a5cd4a130
Number of metabolites,72
Number of reactions,95
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


95 reactions

In [7]:
universal = cobra.Model("universal_reactions")
for i in [i.id for i in x.metabolites.glc__D_e.reactions]:
    reaction = x.reactions.get_by_id(i)
    universal.add_reaction(reaction.copy())
    x.remove_reactions([reaction])

In [8]:
universal.reactions

[<Reaction GLCpts at 0x7f9a5e994550>, <Reaction EX_glc__D_e at 0x7f9a5e994100>]

In [9]:
x

Name,e_coli_core
Memory address,0x07f9a5cd4a130
Number of metabolites,72
Number of reactions,93
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


93 reactions; GLCpts and EX_glc__D_e have been removed from the model.

In [12]:
add_exchange_reactions(x, universal)

Name,e_coli_core
Memory address,0x07fdc8827f040
Number of metabolites,72
Number of reactions,94
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


Unlike GLCpts, EX_glc__D_e has been added to the model.

In [13]:
x.reactions.EX_glc__D_e

Reaction identifier,EX_glc__D_e
Name,D-Glucose exchange
Memory address,0x07fdd21f64eb0
Stoichiometry,glc__D_e <=> D-Glucose <=>
GPR,
Lower bound,-10.0
Upper bound,1000.0


#### HOMOLOGY GAP FILLING FUNCTION

In [27]:
def homology_gapfilling(model, templates, model_obj = None, template_obj = None, use_all_templates = False,
                       integer_threshold = 1e-6, force_exchange = False):
    """
    Performs gap filling on a model using homology models as templates.
    """
    model = load_query_model(model, obj = model_obj)
    model.solver = 'gurobi'
    templates = load_template_models(templates, obj = template_obj)
    # this dict will store used models, genes and reactions
    added_reactions = {}
    # initial flux value
    value = model.optimize().objective_value
    if value == None:
        value = 0.0
    if use_all_templates == False:
        for template in templates:
            template.solver = 'gurobi'
            # result variable will store the reactions ids
            result = gapfilling(model, template, integer_threshold = integer_threshold)
            # log
            log = []
            for reaction in result[0]:
                if reaction.id.startswith("EX_"):
                    log.append((reaction.id, "Exchange reaction"))
                else:
                    log.append((reaction.id, [str(list(reaction.genes)[i]) 
                                              for i in range(len(list(reaction.genes)))]))
            added_reactions[str(template)] = log
            # Adding reactions to the model
            [model.add_reaction(reaction.copy()) for reaction in result[0]]
            # Flux will be evaluated here
            new_value = model.optimize().objective_value
            if new_value != None and new_value > value:
                value = new_value
            elif new_value == None:
                continue
            elif new_value != None and new_value == value:
                break
        return model, added_reactions
    else:
        for template in templates:
            template.solver = 'gurobi'
            result = gapfilling(model, template, integer_threshold = integer_threshold)
            log = []
            for reaction in result[0]:
                if reaction.id.startswith("EX_"):
                    log.append((reaction.id, "Exchange reaction"))
                else:
                    log.append((reaction.id, [str(list(reaction.genes)[i]) 
                                              for i in range(len(list(reaction.genes)))]))
            added_reactions[str(template)] = log
            [model.add_reaction(reaction.copy()) for reaction in result[0]]
        return model, added_reactions

#### LOADING MODELS

In [10]:
# Pseudomonas putida KT2440
iJN746 = read_sbml_model("BiGG_files/iJN746.xml")
iJN746

Name,iJN746
Memory address,0x07f9a5e9f1d00
Number of metabolites,907
Number of reactions,1054
Number of groups,0
Objective expression,1.0*BIOMASS_KT_TEMP - 1.0*BIOMASS_KT_TEMP_reverse_d18f7
Compartments,"extracellular space, cytosol, periplasm"


In [11]:
# Extended model
iJN1463 = read_sbml_model("BiGG_files/iJN1463.xml")
iJN1463

Name,iJN1463
Memory address,0x07f9a74b6d7f0
Number of metabolites,2153
Number of reactions,2927
Number of groups,0
Objective expression,1.0*BIOMASS_KT2440_WT3 - 1.0*BIOMASS_KT2440_WT3_reverse_d86d5
Compartments,"cytosol, extracellular space, periplasm"


In [12]:
# Chlamydomonas reinhardtii
iRC1080 = read_sbml_model("BiGG_files/iRC1080.xml")
iRC1080

No objective coefficients in model. Unclear what should be optimized


Name,iRC1080
Memory address,0x07f9a31c0b2e0
Number of metabolites,1706
Number of reactions,2191
Number of groups,0
Objective expression,0
Compartments,"cytosol, mitochondria, chloroplast, flagellum, peroxisome/glyoxysome, nucleus, golgi apparatus, extracellular space, eyespot, thylakoid"


In [13]:
"""
# adding an objective
iRC1080 = load_query_model(iRC1080, obj = "biomass")
"""

'\n# adding an objective\niRC1080 = load_query_model(iRC1080, obj = "biomass")\n'

In [15]:
# Escherichia coli
# Core-reduced
e_coli_core = read_sbml_model("BiGG_files/e_coli_core.xml")
e_coli_core

Name,e_coli_core
Memory address,0x07f99f23c2a30
Number of metabolites,72
Number of reactions,95
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


In [16]:
# Strain C
iEC1344_C = read_sbml_model("BiGG_files/iEC1344_C.xml")
iEC1344_C

Name,iEC1344_C
Memory address,0x07f99f23c2f10
Number of metabolites,1934
Number of reactions,2726
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, periplasm, extracellular space"


In [17]:
# Strain W
iEC1364_W = read_sbml_model("BiGG_files/iEC1364_W.xml")
iEC1364_W

Name,iEC1364_W
Memory address,0x07f99f23eee50
Number of metabolites,1927
Number of reactions,2764
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, extracellular space, periplasm"


In [18]:
# Bacillus subtilis
iYO844 = read_sbml_model("BiGG_files/iYO844.xml")
iYO844 

Name,iYO844
Memory address,0x07f99eaa1f6d0
Number of metabolites,990
Number of reactions,1250
Number of groups,0
Objective expression,1.0*BIOMASS_BS_10 - 1.0*BIOMASS_BS_10_reverse_8788b
Compartments,"cytosol, extracellular space"


In [19]:
# Saccharomyces cerevisiae
iMM904 = read_sbml_model("BiGG_files/iMM904.xml")
iMM904

Name,iMM904
Memory address,0x07f99e56a0970
Number of metabolites,1226
Number of reactions,1577
Number of groups,0
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


In [20]:
iND750 = read_sbml_model("BiGG_files/iND750.xml")
iND750

Name,iND750
Memory address,0x07f99e5682cd0
Number of metabolites,1059
Number of reactions,1266
Number of groups,0
Objective expression,1.0*BIOMASS_SC4_bal - 1.0*BIOMASS_SC4_bal_reverse_bb385
Compartments,"extracellular space, cytosol, mitochondria, peroxisome/glyoxysome, nucleus, golgi apparatus, vacuole, endoplasmic reticulum"


In [21]:
# Mus musculus
iMM1415 = read_sbml_model("BiGG_files/iMM1415.xml")
iMM1415

No objective coefficients in model. Unclear what should be optimized


Name,iMM1415
Memory address,0x07f99e2e74f40
Number of metabolites,2775
Number of reactions,3726
Number of groups,0
Objective expression,0
Compartments,"cytosol, extracellular space, golgi apparatus, lysosome, mitochondria, nucleus, endoplasmic reticulum, peroxisome/glyoxysome"


#### TESTING

In [22]:
# Case 1a: gapfill a simple model with itself (biomass)
e_coli_core2 = read_sbml_model("BiGG_files/e_coli_core.xml")
e_coli_core2

Name,e_coli_core
Memory address,0x07f99e2e74eb0
Number of metabolites,72
Number of reactions,95
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


In [24]:
# moving essential reactions to a new model
universal = cobra.Model("universal_reactions")
for i in [i.id for i in e_coli_core2.metabolites.glc__D_e.reactions]:
    reaction = e_coli_core2.reactions.get_by_id(i)
    universal.add_reaction(reaction.copy())
    e_coli_core2.remove_reactions([reaction])

In [25]:
e_coli_core2.optimize().objective_value

/home/fco/miniconda3/envs/TFM/lib/python3.8/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)


In [28]:
e_coli_core2_gp, reactions1 = homology_gapfilling(e_coli_core2, templates=[universal])

Read LP format model from file /tmp/tmpjzk2vnrf.lp
Reading time = 0.00 seconds
: 72 rows, 186 columns, 710 nonzeros
Read LP format model from file /tmp/tmp6ipsmvy1.lp
Reading time = 0.00 seconds
: 72 rows, 186 columns, 710 nonzeros
Read LP format model from file /tmp/tmpjx8m4_2l.lp
Reading time = 0.00 seconds
: 4 rows, 4 columns, 10 nonzeros
Read LP format model from file /tmp/tmpsa594zmj.lp
Reading time = 0.00 seconds
: 4 rows, 4 columns, 10 nonzeros


In [29]:
reactions1

{'universal_reactions': [('EX_glc__D_e', 'Exchange reaction'),
  ('GLCpts',
   ['b1819', 'b1817', 'b2416', 'b2417', 'b2415', 'b1101', 'b1621', 'b1818'])]}

In [30]:
e_coli_core2_gp.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,4.765,0,0.00%
o2_e,EX_o2_e,21.8,0,0.00%
pi_e,EX_pi_e,3.215,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-22.81,1,100.00%
h2o_e,EX_h2o_e,-29.18,0,0.00%
h_e,EX_h_e,-17.53,0,0.00%


In [31]:
# Case 1b: gapfill a model with itself (reaction)
e_coli_core3 = read_sbml_model("BiGG_files/e_coli_core.xml")

In [32]:
e_coli_core3.reactions.CO2t

Reaction identifier,CO2t
Name,CO2 transporter via diffusion
Memory address,0x07f99ca31d9d0
Stoichiometry,co2_e <=> co2_c CO2 CO2 <=> CO2 CO2
GPR,s0001
Lower bound,-1000.0
Upper bound,1000.0


In [33]:
# We set an objective other than biomass
e_coli_core3 = load_query_model(e_coli_core3, obj='CO2t')

In [34]:
e_coli_core3.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,11.1,1,15.62%
glc__D_e,EX_glc__D_e,10,6,84.38%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-4.479,2,12.60%
for_e,EX_for_e,-8.896,1,12.51%
h2o_e,EX_h2o_e,-2.208,0,0.00%
h_e,EX_h_e,-40,0,0.00%
succ_e,EX_succ_e,-13.31,4,74.89%


In [35]:
# we will eliminate reactions which don't contain co2_e or co2_c
universal = cobra.Model("universal_reactions")
for i in [i.id for i in e_coli_core3.metabolites.succ_e.reactions]:
    reaction = e_coli_core3.reactions.get_by_id(i)
    universal.add_reaction(reaction.copy())
    e_coli_core3.remove_reactions([reaction])

In [36]:
e_coli_core3.optimize().objective_value

0.7901367591711717

In [37]:
universal.reactions

[<Reaction EX_succ_e at 0x7f99ca2e6a00>,
 <Reaction SUCCt2_2 at 0x7f9a31a69e20>,
 <Reaction SUCCt3 at 0x7f99ca32be50>]

In [38]:
e_coli_core3.optimize()

,fluxes,reduced_costs
PFK,9.560111,1.561251e-17
PFL,15.413253,0.000000e+00
PGI,9.909342,0.000000e+00
PGK,-18.824413,-0.000000e+00
PGL,0.000000,0.000000e+00
...,...,...
NADH16,12.810450,0.000000e+00
NADTRHD,0.000000,-1.530976e-02
NH4t,2.411405,0.000000e+00
O2t,6.405225,0.000000e+00


In [40]:
# standard procedure
e_coli_core3_gp, reactions2 = homology_gapfilling(e_coli_core3, templates = [universal], template_obj='CO2t')

['universal_reactions']
Read LP format model from file /tmp/tmpiwj6h6nj.lp
Reading time = 0.00 seconds
: 72 rows, 184 columns, 702 nonzeros
Read LP format model from file /tmp/tmpqto6feca.lp
Reading time = 0.00 seconds
: 72 rows, 184 columns, 702 nonzeros
Read LP format model from file /tmp/tmp1nvi68kt.lp
Reading time = 0.00 seconds
: 4 rows, 6 columns, 18 nonzeros
Read LP format model from file /tmp/tmpdvzryhgx.lp
Reading time = 0.00 seconds
: 4 rows, 6 columns, 18 nonzeros


In [42]:
e_coli_core3_gp.optimize().objective_value

0.7901367591711717

In [43]:
reactions2

{'universal_reactions': []}

It doesn't add the succinate reactions although they would improve the flux

In [47]:
# forcing succinate addition
with e_coli_core3:
    e_coli_core3.objective = e_coli_core3.add_boundary(e_coli_core3.metabolites.succ_e, type='demand')
    solution = gapfill(e_coli_core3, universal)
    print([reaction for reaction in solution[0]])
    [e_coli_core3.add_reaction(reaction.copy()) for reaction in solution[0]]

Read LP format model from file /tmp/tmpe5wy0y0e.lp
Reading time = 0.00 seconds
: 72 rows, 186 columns, 704 nonzeros
Read LP format model from file /tmp/tmpx8fx4d2u.lp
Reading time = 0.00 seconds
: 72 rows, 186 columns, 704 nonzeros
Read LP format model from file /tmp/tmpgp7rnrqb.lp
Reading time = 0.00 seconds
: 4 rows, 6 columns, 18 nonzeros
Read LP format model from file /tmp/tmpl1v5ihwy.lp
Reading time = 0.00 seconds
: 4 rows, 6 columns, 18 nonzeros
[<Reaction SUCCt3 at 0x7f99ca069a90>]


In [48]:
e_coli_core3.optimize().objective_value

0.7901367591711717

No reactions have been added from universal; it is a viable model but it should have add them to maximize fluxes

Another case is the following: removing certain reactions from a model, for example EX_CO2, and try to optimize that reaction. That gap filling would be infeasible. 

In [49]:
# Case 1c: gapfill a complex model with itself (biomass)
iMM904.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,1.611,0,0.00%
o2_e,EX_o2_e,2,0,0.00%
pi_e,EX_pi_e,0.05691,0,0.00%
so4_e,EX_so4_e,0.02225,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-18.02,1,36.29%
etoh_e,EX_etoh_e,-15.82,2,63.70%
for_e,EX_for_e,-0.001488,1,0.00%
h2o_e,EX_h2o_e,-5.64,0,0.00%


In [51]:
universal = cobra.Model("universal_reactions")
for i in [i.id for i in iMM904.metabolites.g6p_r.reactions]:
    reaction = iMM904.reactions.get_by_id(i)
    universal.add_reaction(reaction.copy())
    iMM904.remove_reactions([reaction])

In [52]:
iMM904.optimize().objective_value

0.0

In [53]:
universal.reactions

[<Reaction G6Pter at 0x7f99c9fec730>, <Reaction G6PDH2er at 0x7f99ca3c3040>]

In [54]:
iMM904_gp, reactions3 = homology_gapfilling(iMM904, [universal])

Read LP format model from file /tmp/tmpe0dkybs5.lp
Reading time = 0.01 seconds
: 1226 rows, 3150 columns, 13020 nonzeros
Read LP format model from file /tmp/tmpkhu4rs1a.lp
Reading time = 0.01 seconds
: 1226 rows, 3150 columns, 13020 nonzeros
Read LP format model from file /tmp/tmp7uyolxix.lp
Reading time = 0.00 seconds
: 6 rows, 4 columns, 14 nonzeros
Read LP format model from file /tmp/tmpva1zdalf.lp
Reading time = 0.00 seconds
: 6 rows, 4 columns, 14 nonzeros


RuntimeError: failed to validate gapfilled model, try lowering the integer_threshold

This time the error is related to the integer threshold of the Gapfiller class.

In [55]:
# lowering integer threshold
iMM904_gp, reactions3 = homology_gapfilling(iMM904, [universal], integer_threshold=1e-09)

Read LP format model from file /tmp/tmpmxu0hi8e.lp
Reading time = 0.02 seconds
: 1226 rows, 3150 columns, 13020 nonzeros
Read LP format model from file /tmp/tmpkgmzb1kn.lp
Reading time = 0.01 seconds
: 1226 rows, 3150 columns, 13020 nonzeros
Read LP format model from file /tmp/tmpa4tr_1tv.lp
Reading time = 0.00 seconds
: 6 rows, 4 columns, 14 nonzeros
Read LP format model from file /tmp/tmpnosov7mq.lp
Reading time = 0.00 seconds
: 6 rows, 4 columns, 14 nonzeros


RuntimeError: failed to validate gapfilled model, try lowering the integer_threshold

Integer threshold must be lowered but 1e-9 is the minimum for gurobi

In [57]:
# adding exchange reactions
iMM904_gp, reactions3 = homology_gapfilling(iMM904, [universal], force_exchange=True, integer_threshold = 1e-09)

Read LP format model from file /tmp/tmp7hgxjn1q.lp
Reading time = 0.01 seconds
: 1226 rows, 3150 columns, 13020 nonzeros
Read LP format model from file /tmp/tmp05ujvivn.lp
Reading time = 0.01 seconds
: 1226 rows, 3150 columns, 13020 nonzeros
Read LP format model from file /tmp/tmppzd_x65l.lp
Reading time = 0.00 seconds
: 6 rows, 4 columns, 14 nonzeros
Read LP format model from file /tmp/tmpb42i25dj.lp
Reading time = 0.00 seconds
: 6 rows, 4 columns, 14 nonzeros


RuntimeError: failed to validate gapfilled model, try lowering the integer_threshold

In [58]:
# Case 2a: Using other models as templates
e_coli_core4 = read_sbml_model("BiGG_files/e_coli_core.xml")

In [59]:
e_coli_core4.optimize().objective_value

0.8739215069684302

In [60]:
t = load_template_models([iEC1344_C, iEC1364_W])

In [61]:
e_coli_core4_gp, reactions4 = homology_gapfilling(e_coli_core4, templates = t, use_all_templates=True)

Read LP format model from file /tmp/tmplbd23_kc.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmp1_uv16j3.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmp7i68rp8r.lp
Reading time = 0.01 seconds
: 1934 rows, 5452 columns, 21416 nonzeros
Read LP format model from file /tmp/tmpdyxkn0ud.lp
Reading time = 0.04 seconds
: 1934 rows, 5452 columns, 21416 nonzeros
Read LP format model from file /tmp/tmpixb_2gka.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmp4majy9br.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmpvl39n12y.lp
Reading time = 0.01 seconds
: 1927 rows, 5528 columns, 21688 nonzeros
Read LP format model from file /tmp/tmpka_8yf53.lp
Reading time = 0.01 seconds
: 1927 rows, 5528 columns, 21688 nonzeros


In [62]:
e_coli_core4_gp.optimize().objective_value

0.8739215069684302

In [63]:
reactions4

{'iEC1344_C': [], 'iEC1364_W': []}

No reactions added. That could mean that the core model has all the needed reactions for an optimal biomass flux.

In [64]:
# Case 2b: using more models as templates
e_coli_core5 = read_sbml_model("BiGG_files/e_coli_core.xml")

In [65]:
t = load_template_models([iEC1344_C, iEC1364_W, iJN1463, iJN746, iMM904, iND750])

In [66]:
e_coli_core5_gp, reactions5 = homology_gapfilling(e_coli_core5, t, use_all_templates=True, integer_threshold = 1e-9)

Read LP format model from file /tmp/tmpcfwceue7.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmpn9mtslh5.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmpiuxu60ul.lp
Reading time = 0.01 seconds
: 1934 rows, 5452 columns, 21416 nonzeros
Read LP format model from file /tmp/tmpnyq5q0w0.lp
Reading time = 0.01 seconds
: 1934 rows, 5452 columns, 21416 nonzeros
Read LP format model from file /tmp/tmpk8e6c5kr.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmp45grzsfz.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmp4fmtjq9p.lp
Reading time = 0.01 seconds
: 1927 rows, 5528 columns, 21688 nonzeros
Read LP format model from file /tmp/tmp25wcc0im.lp
Reading time = 0.02 seconds
: 1927 rows, 5528 columns, 21688 nonzeros
Read LP format model from file /tmp/tmpmjgnqxg1.lp
R

Infeasible: gapfilling optimization failed (infeasible)

In [67]:
# trying with simpler models
t = load_template_models([iEC1344_C, iEC1364_W, iJN1463, iJN746, iYO844])

In [68]:
e_coli_core5_gp, reactions5 = homology_gapfilling(e_coli_core5, t, use_all_templates=True, integer_threshold = 1e-9)

Read LP format model from file /tmp/tmpd0zeqa92.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmpp1xhty95.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmpvhly7gy3.lp
Reading time = 0.01 seconds
: 1934 rows, 5452 columns, 21416 nonzeros
Read LP format model from file /tmp/tmp3zffch4e.lp
Reading time = 0.01 seconds
: 1934 rows, 5452 columns, 21416 nonzeros
Read LP format model from file /tmp/tmpo59sqxwj.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmpwxm5o8em.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmppszsogc5.lp
Reading time = 0.01 seconds
: 1927 rows, 5528 columns, 21688 nonzeros
Read LP format model from file /tmp/tmp0p902z17.lp
Reading time = 0.01 seconds
: 1927 rows, 5528 columns, 21688 nonzeros
Read LP format model from file /tmp/tmpo3awv_qj.lp
R

Infeasible: gapfilling optimization failed (infeasible)

In [69]:
# Case 2c: using other models as templates (reaction)
e_coli_core6 = load_query_model(e_coli_core, obj='CO2t')
e_coli_core6.optimize().objective_value

11.104242424242424

In [70]:
e_coli_core6_gp, reactions6 = homology_gapfilling(e_coli_core6, t, use_all_templates=True, integer_threshold=1e-9)

Read LP format model from file /tmp/tmprm4tymvp.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmpn0xt7cdh.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmp1kwpjhck.lp
Reading time = 0.01 seconds
: 1934 rows, 5452 columns, 21416 nonzeros
Read LP format model from file /tmp/tmp2ez4oeq8.lp
Reading time = 0.01 seconds
: 1934 rows, 5452 columns, 21416 nonzeros
Read LP format model from file /tmp/tmpl4jfuzus.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmpt1g6qgo1.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmp3oodpnhp.lp
Reading time = 0.01 seconds
: 1927 rows, 5528 columns, 21688 nonzeros
Read LP format model from file /tmp/tmpayi2hi6y.lp
Reading time = 0.01 seconds
: 1927 rows, 5528 columns, 21688 nonzeros
Read LP format model from file /tmp/tmp23hcxqjd.lp
R

Infeasible: gapfilling optimization failed (infeasible)